<a href="https://colab.research.google.com/github/AvantiShri/gcp_analysis/blob/main/alldata/ComputeLifetimeDeviceStats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#mount Google drive to download the data some

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
%cd /content/drive/MyDrive/GCP_data/all_data
#data at https://drive.google.com/drive/u/0/folders/1MUS-xbwoBiodWLz4gese19ti_ftlKsVu

/content/drive/MyDrive/GCP_data/all_data


In [19]:
#install some utilities code at https://github.com/AvantiShri/gcpdatautils
!pip uninstall -y gcpdatautils #uninstall the pre-existing version if want to fetch latest
!pip install git+https://github.com/AvantiShri/gcpdatautils.git #install latest from github

#Note: the code that was used to consolidate the downloaded csv.gz files into hdf5 files
# is at https://github.com/AvantiShri/gcp_analysis/blob/main/alldata/Consolidate_GCP_as_HDF5.ipynb

Found existing installation: gcpdatautils 0.1.1.0
Uninstalling gcpdatautils-0.1.1.0:
  Successfully uninstalled gcpdatautils-0.1.1.0
  Cloning https://github.com/AvantiShri/gcpdatautils.git to /tmp/pip-req-build-sc9979u1
  Running command git clone --filter=blob:none --quiet https://github.com/AvantiShri/gcpdatautils.git /tmp/pip-req-build-sc9979u1
  Resolved https://github.com/AvantiShri/gcpdatautils.git to commit d101c1b7b435d3e0878e5f635943d4479fa76267
  Preparing metadata (setup.py) ... done
  Created wheel for gcpdatautils: filename=gcpdatautils-0.1.1.0-py3-none-any.whl size=6390 sha256=11c2efb04b130d53bdc83e6f20589718d7c1b9965f1247eb0a7b9587dedd5286
  Stored in directory: /tmp/pip-ephem-wheel-cache-ah5iv2__/wheels/fe/f9/b7/d5afa50c9e111dcf3a66edb57f0c2273e54d108591664596b8
Successfully built gcpdatautils


In [20]:
from gcpdatautils import GCPHdf5DataReader

gcp_data_reader = GCPHdf5DataReader()

Parsing the bad data file: /usr/local/lib/python3.10/dist-packages/gcpdatautils/resources/rotteneggs.txt
manually correcting 47,2000-01-01 00:00:00,2001-06-31 23:59:59,2222
manually correcting 47,2008-04-25 00:00:00,2008-04-55 23:59:59,1092


In [ ]:
from datetime import datetime
from collections import defaultdict
import numpy as np

device_to_numtrials = defaultdict(lambda: 0)
device_to_trialsums = defaultdict(lambda: 0)
device_to_trialsquaredsums = defaultdict(lambda: 0)

for year in range(1998, 2024):
  days = gcp_data_reader.get_available_days_in_year(year)
  for day in days:
    print(day)
    year_fh = gcp_data_reader.get_fh_for_year(year) #get the file handle for the year
    #check the metadata for the start and end timestamps of the day (some days have incomplete data)
    start_timestamp = year_fh[day].attrs["start_time"]
    end_timestamp = year_fh[day].attrs["end_time"]
    day_data, day_devices = gcp_data_reader.fetch_data_within_day(
          starttime=datetime.fromtimestamp(start_timestamp),
          endtime=datetime.fromtimestamp(end_timestamp),
          bail_if_missing_seconds=False,
          mask_bad_data=True) #if we do not mask bad data, some devices (e.g. 223) end up with v. low variance (https://github.com/AvantiShri/gcp_analysis/blob/024fc4c205126b3eb7ebc037b91de3b6859b10f7/alldata/ComputeLifetimeDeviceStats.ipynb)
                               #but those devices are also mostly flagged as bad data
    for i, device in enumerate(day_devices):
      device_to_numtrials[device] += np.sum(np.isnan(day_data[:,i])==False)
      device_to_trialsums[device] += np.nansum(day_data[:,i])
      device_to_trialsquaredsums[device] += np.nansum(np.square(day_data[:,i]))

Streaming output truncated to the last 5000 lines.
2005-01-18
2005-01-19
2005-01-20
2005-01-21
2005-01-22
2005-01-23
2005-01-24
2005-01-25
2005-01-26
2005-01-27
2005-01-28
2005-01-29
2005-01-30
2005-01-31
2005-02-01
2005-02-02
2005-02-03
2005-02-04
2005-02-05
2005-02-06
2005-02-07
2005-02-08
2005-02-09
2005-02-10
2005-02-11
2005-02-12
2005-02-13
2005-02-14
2005-02-15
2005-02-16
2005-02-17
Before masking, fraction of nans in raw data is 0.06631365740740741 for 2005-02-17 00:00:00 to 2005-02-17 23:59:59
Found 'rotten egg' entries for device id: 2023 in time range 2005-02-17 00:00:00 to 2005-02-17 23:59:59 (range: 2005-02-17 00:00:00 to 2005-02-17 23:59:59 )
After masking, fraction of nans in raw data is 0.07305704898446834
2005-02-18
2005-02-19
2005-02-20
2005-02-21
2005-02-22
2005-02-23
2005-02-24
2005-02-25
2005-02-26
2005-02-27
2005-02-28
2005-03-01
2005-03-02
2005-03-03
2005-03-04
2005-03-05
2005-03-06
2005-03-07
2005-03-08
2005-03-09
2005-03-10
2005-03-11
2005-03-12
2005-03-13
2005-

In [15]:
device_to_empirical_means = {}
device_to_empirical_variances = {}

for device in sorted(device_to_numtrials.keys()):
  device_to_empirical_means[int(device)] = device_to_trialsums[device]/device_to_numtrials[device]
  device_to_empirical_variances[int(device)] = device_to_trialsquaredsums[device]/device_to_numtrials[device] - np.square(device_to_empirical_means[device])
  print(device, device_to_empirical_means[device], device_to_empirical_variances[device], device_to_numtrials[device])

1 100.00003504010576 50.039383927110975 757046802
28 99.99947194999292 50.020409149625266 331093642
33 99.99905513535508 49.98364548077734 73111001
34 99.9991490545327 50.010904383181696 16070360
37 99.99951231229896 49.97856402159414 764667633
100 100.00063784230353 50.03458648378182 186086748
101 99.99902822615114 50.02586081652407 232433709
102 99.99985982160378 50.0199165394406 299461266
103 100.00037224849079 49.999872650772886 579102415
104 100.11563462196374 50.08789445698858 2923
105 99.99986369053155 50.035604185643024 350745994
106 99.99947275333265 50.03929341236653 258548813
107 99.99975286590929 50.0261066558287 80539273
108 100.00020286871533 50.03925770446949 443528219
109 99.99979810292754 50.05367296222721 81313710
110 99.99856821619765 50.02760211901841 513774495
111 100.00003193696979 50.02271049200499 525472520
112 99.99991943420132 49.98001778680009 728038460
114 99.99970929893038 50.02288781737661 166359897
115 100.00026228187825 50.074980987126764 266244090
116 9

<ipython-input-15-946aa19d8983>:5: RuntimeWarning: invalid value encountered in double_scalars
  device_to_empirical_means[int(device)] = device_to_trialsums[device]/device_to_numtrials[device]
<ipython-input-15-946aa19d8983>:6: RuntimeWarning: invalid value encountered in double_scalars
  device_to_empirical_variances[int(device)] = device_to_trialsquaredsums[device]/device_to_numtrials[device] - np.square(device_to_empirical_means[device])


In [16]:
import json

with open("empirical_device_variances.json", 'w') as f:
  f.write(json.dumps(device_to_empirical_variances))

with open("empirical_device_means.json", 'w') as f:
  f.write(json.dumps(device_to_empirical_means))